# Workflow registry authorization

In [2]:
# set the lifemonitor root
lifemonitor_root = "/mnt/data/projects/@crs4/EOSC-Life/Repositories/life_monitor_rachk8s"
%cd {lifemonitor_root}

/mnt/data/projects/@crs4/EOSC-Life/Repositories/life_monitor_rachk8s


In [9]:
# import requirements 
import requests
import pprint
pp = pprint.PrettyPrinter(indent=2).pprint

In [6]:
# list registries
!docker-compose exec lm /bin/bash -c "flask registry show seek"

INFO:lifemonitor.app:Logging is active. Log level: DEBUG


****************************************************************************************************
Workflow Registry 'seek' (uuid: 1c40002b-8676-4f51-b529-9ff02ff78234, type: seek) registered!
****************************************************************************************************


OAuth2 settings to connect to LifeMonitor:
----------------------------------------------------------------------------------------------------
REGISTRY NAME: seek
REGISTRY API URL: https://seek:3000
REGISTRY CLIENT ID: bAVlsOwx2Z7sPgQJU7mdg1CE
REGISTRY CLIENT SECRET: FQ8FE57DvYy9C5NKsAgXaBXAQ061gsNXz71X8I8radX6zHU2
REGISTRY CLIENT ALLOWED SCOPES: read write
REGISTRY CLIENT ALLOWED FLOWS: ['client_credentials', 'authorization_code', 'refresh_token']
REGISTRY CLIENT REDIRECT URIs: ['https://seek:3000']
REGISTRY CLIENT AUTH METHOD: client_secret_post
AUTHORIZE URL: <LIFE_MONITOR_BASE_URL>/oauth2/authorize/seek
ACCESS TOKEN URL: <LIFE_MON

In [10]:
# Set Registry Credentials from LifeMonitor
CLIENT_ID = "bAVlsOwx2Z7sPgQJU7mdg1CE"
CLIENT_SECRET = "FQ8FE57DvYy9C5NKsAgXaBXAQ061gsNXz71X8I8radX6zHU2"

In [11]:
# HTTP settings to connect to LifeMonitor

s = requests.session() # actually not mandatory, but just to share some settings among requests

# if you use self-signed certificates,
# you have to uncomment the line below to disable SSL verification
s.verify = False

# common header settings
s.headers.update({})

In [12]:
# LifeMonitor URLs
lm_base_url = "https://lm:8443"
lm_token_url = f"{lm_base_url}/oauth2/token"

In [14]:
# Get an authorization token from LifeMonitor
token_response = s.post(
    lm_token_url, 
    data={
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "grant_type": "client_credentials",
        "scope": "read write"
    }, allow_redirects=True, verify=False)
assert token_response.status_code == 200, "OAuth2 Error"
token = token_response.json()
pp(token)

{ 'access_token': '1WHYfVkZBsiwf55DQ9XGKSiA1eUWxJ7HPtVlwAvm4d',
  'expires_in': 864000,
  'scope': 'read write',
  'token_type': 'Bearer'}


In [15]:
# Update headers with the OAuth2 token
s.headers.update({'Authorization': f"Bearer {token['access_token']}"})

In [17]:
# Get registry info
response = s.get(f"{lm_base_url}/registries/current")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
current_registry = response.json()
pp(current_registry)

{ 'name': 'seek',
  'type': 'seek',
  'uri': 'https://seek:3000',
  'uuid': '1c40002b-8676-4f51-b529-9ff02ff78234'}
